In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import os
from keras.models import *
from support_function import *


2023-04-19 16:22:05.213808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = load_model('/Users/parthkalathia/Desktop/CNN_SCRATCH/MODELS/3dcnn_v2-2.h5')

2023-04-19 16:22:25.280368: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
import numpy as np    # for mathematical operations
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import os

import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from moviepy.editor import *
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

image_height, image_width = 128, 128
max_images_per_class = 800

dataset_directory = '/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2'

def make_average_predictions(video_file_path, predictions_frames_count, model):
    
    # Initializing the Numpy array which will store Prediction Probabilities
    predicted_labels_probabilities_np = np.zeros((predictions_frames_count, model_output_size), dtype = np.float)

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)
    print(video_file_path)
    # Getting The Total Frames present in the video 
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculating The Number of Frames to skip Before reading a frame
    skip_frames_window = video_frames_count // predictions_frames_count
    
    all_frame_predictions_proability = {}
    for frame_counter in range(predictions_frames_count): 

        # Setting Frame Position
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading The Frame
        _ , frame = video_reader.read() 

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        resized_frame= resized_frame.reshape(-1, image_height, image_width,3)
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]

        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_np[frame_counter] = predicted_labels_probabilities

        all_frame_predictions_proability[frame_counter] = predicted_labels_probabilities
        
    # Calculating Average of Predicted Labels Probabilities Column Wise 
    predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)

    # Sorting the Averaged Predicted Labels Probabilities
    predicted_labels_probabilities_averaged_sorted_indexes = np.argsort(predicted_labels_probabilities_averaged)[::-1]

    # class_name = []
    # predicted_class_prebability = []
    predicted_dict = {}
    # Iterating Over All Averaged Predicted Label Probabilities
    for predicted_label in predicted_labels_probabilities_averaged_sorted_indexes:

        # Accessing The Class Name using predicted label.
        predicted_class_name = classes_list[predicted_label]

        # Accessing The Averaged Probability using predicted label.
        predicted_probability = predicted_labels_probabilities_averaged[predicted_label]
        predicted_dict[predicted_class_name] = predicted_probability
        
        print(predicted_probability)
       # print(f"CLASS NAME: {predicted_class_name}   AVERAGED PROBABILITY: {(predicted_probability*100):.2}")
    
    # Closing the VideoCapture Object and releasing all resources held by it. 
    video_reader.release()

    return predicted_dict, all_frame_predictions_proability

In [4]:
test_files_list = os.listdir('/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL')
predictions = []
test_files_list.remove('.DS_Store')
test_files_list.remove('T49.jpg')
test_files_list.remove('M4.mp4')
test_files_list.remove('T12.mp4')
test_files_list.remove('T16.mp4')
test_files_list.remove('T47.mp4')
test_files_list.remove('T48.mp4')
test_files_list.remove('T69.mp4')
test_files_list.remove('T70.mp4')
num_of_frame = 50 
for test_file in test_files_list:
    input_video_file_path = f'/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/{test_file}'
    predictions.append([test_file, make_average_predictions(input_video_file_path, num_of_frame, model)])

/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/T20.mp4


ValueError: in user code:

    File "/Users/parthkalathia/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/Users/parthkalathia/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/parthkalathia/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/parthkalathia/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/Users/parthkalathia/opt/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/parthkalathia/opt/anaconda3/lib/python3.9/site-pack

In [131]:
# video vise all frame probabilities
all_frame_data = pd.DataFrame(columns=['Video', 'Frame No.', 'TALKING', 'HEADDOWN', 'YAWNING', 'MOBILE', 'NORMAL'])
for i in range(len(predictions)):
    
    for j in range(num_of_frame):
        if j==0:
            data_row = [predictions[i][0]]
        else:
            data_row = [""]
        data_row.append(f'frame_{j+1}')
        for k in range(5):
            data_row.append(predictions[i][1][1][j][k])
        print(data_row)
        all_frame_data.loc[len(all_frame_data)] = data_row
    
    all_frame_data.loc[len(all_frame_data)] = ['', '', '', '', '', '', '']
    predictions[i][1][0]['Video'] = ''
    predictions[i][1][0]['Frame No.'] = ''
    all_frame_data.loc[len(all_frame_data)] = predictions[i][1][0]
    all_frame_data.loc[len(all_frame_data)] = ['', '', '', '', '', '', '']
    
all_frame_data.to_csv('ED2_3DCNN_TEST_frame_10.csv')        

['T20.mp4', 'frame_1', 0.005358227, 0.000824451, 0.00016978125, 0.38501066, 0.6086369]
['', 'frame_2', 0.007858221, 0.0009885189, 0.00018258161, 0.31073615, 0.68023455]
['', 'frame_3', 0.024637222, 0.0011528413, 6.260998e-05, 0.46433595, 0.50981134]
['', 'frame_4', 0.032241255, 0.000685451, 3.4561424e-05, 0.6248509, 0.34218782]
['', 'frame_5', 0.0566233, 0.00040089333, 1.9878531e-05, 0.56483585, 0.37811998]
['', 'frame_6', 0.068107836, 0.00079694914, 2.5658232e-05, 0.2562619, 0.67480767]
['', 'frame_7', 0.13535921, 0.001112942, 6.4682215e-05, 0.26391226, 0.59955096]
['', 'frame_8', 0.18767539, 0.001847396, 0.00015875342, 0.36143494, 0.44888347]
['', 'frame_9', 0.08418786, 0.001712864, 0.00025628976, 0.43243194, 0.481411]
['', 'frame_10', 0.023530079, 0.0017085082, 0.000114748225, 0.21883184, 0.7558149]
['', 'frame_11', 0.017979613, 0.0007148543, 0.00015489369, 0.2946631, 0.6864876]
['', 'frame_12', 0.021874089, 0.0007596803, 0.00016384551, 0.34985492, 0.62734747]
['', 'frame_13', 0.039

['', 'frame_42', 0.0047544353, 0.97930294, 6.753907e-05, 1.3754043e-05, 0.01586125]
['', 'frame_43', 0.0040805973, 0.9822636, 9.9297155e-05, 1.1845506e-05, 0.01354457]
['', 'frame_44', 0.0037519685, 0.9933895, 2.565004e-05, 6.6566145e-06, 0.0028261933]
['', 'frame_45', 0.0016694355, 0.99696535, 2.1356282e-05, 4.591024e-06, 0.0013392114]
['', 'frame_46', 0.0002957165, 0.99816954, 6.508168e-06, 2.2094287e-06, 0.0015259762]
['', 'frame_47', 0.00031146838, 0.9989975, 3.922027e-06, 2.5587651e-06, 0.0006847004]
['', 'frame_48', 0.0057737837, 0.99378514, 5.085959e-06, 9.099359e-06, 0.00042679007]
['', 'frame_49', 0.00051348784, 0.99904424, 5.499959e-06, 3.4306315e-06, 0.00043324794]
['', 'frame_50', 0.0015118528, 0.9950781, 7.423033e-05, 8.705747e-06, 0.003327155]
['T18.mp4', 'frame_1', 2.410867e-08, 1.7742705e-05, 3.177391e-07, 1.7846995e-10, 0.9999819]
['', 'frame_2', 2.4205818e-07, 5.882629e-05, 7.562e-07, 6.535153e-09, 0.99994016]
['', 'frame_3', 2.502729e-08, 8.396227e-06, 3.4300882e-07,

['T68.mp4', 'frame_1', 0.025725413, 0.6744029, 0.00083514, 0.0010011807, 0.29803544]
['', 'frame_2', 0.023885416, 0.64930964, 0.000836565, 0.0010534682, 0.32491493]
['', 'frame_3', 0.022182098, 0.6533484, 0.0008226611, 0.0009826758, 0.32266408]
['', 'frame_4', 0.039554708, 0.6096506, 0.00089904596, 0.0013141863, 0.34858146]
['', 'frame_5', 0.05702893, 0.5456739, 0.0007911901, 0.0018073028, 0.39469865]
['', 'frame_6', 0.015490199, 0.40101194, 0.00035527753, 0.0014960952, 0.5816464]
['', 'frame_7', 0.02190962, 0.47292772, 0.00021842943, 0.0013433639, 0.50360084]
['', 'frame_8', 0.0380621, 0.33735642, 0.0002913139, 0.0021103716, 0.6221798]
['', 'frame_9', 0.030006122, 0.18325047, 0.0003835002, 0.005462161, 0.78089774]
['', 'frame_10', 0.014606916, 0.14604354, 0.00019006783, 0.0082030175, 0.8309565]
['', 'frame_11', 0.01616697, 0.4474625, 0.00086446595, 0.008374949, 0.5271311]
['', 'frame_12', 0.025266787, 0.62501323, 0.001681727, 0.022458777, 0.32557952]
['', 'frame_13', 0.020173807, 0.70

['', 'frame_4', 0.0003676202, 2.6172987e-07, 2.155644e-06, 0.99962187, 8.067681e-06]
['', 'frame_5', 0.0002992339, 3.9588346e-07, 2.67541e-06, 0.99969065, 7.1649115e-06]
['', 'frame_6', 0.00017925032, 6.227167e-07, 2.7894096e-06, 0.99981195, 5.4468446e-06]
['', 'frame_7', 6.775383e-05, 5.2350754e-07, 2.1223013e-06, 0.9999263, 3.3208653e-06]
['', 'frame_8', 4.00832e-05, 1.5976978e-07, 1.3605307e-06, 0.9999565, 1.9397733e-06]
['', 'frame_9', 1.870592e-05, 3.988984e-08, 4.2314056e-07, 0.99998045, 4.9111526e-07]
['', 'frame_10', 1.0180346e-05, 2.660338e-08, 1.832985e-07, 0.9999894, 2.3520711e-07]
['', 'frame_11', 8.697162e-06, 1.8697012e-08, 1.3450709e-07, 0.99999106, 1.2059863e-07]
['', 'frame_12', 7.5171765e-06, 2.0802226e-08, 1.3572364e-07, 0.99999225, 7.214046e-08]
['', 'frame_13', 5.0019817e-06, 1.1209687e-08, 8.8544994e-08, 0.9999949, 3.3731574e-08]
['', 'frame_14', 3.763389e-06, 1.9744226e-09, 1.7228581e-07, 0.99999607, 1.0039384e-08]
['', 'frame_15', 2.885606e-06, 7.218477e-10, 2.1

['', 'frame_14', 0.0010767237, 0.36421624, 0.42295536, 0.16004689, 0.05170485]
['', 'frame_15', 0.001116576, 0.3793463, 0.2295029, 0.30279952, 0.08723465]
['', 'frame_16', 0.0007156129, 0.3133168, 0.022817517, 0.55120164, 0.11194841]
['', 'frame_17', 0.0006119367, 0.08697151, 0.0033801447, 0.7994071, 0.10962928]
['', 'frame_18', 0.00025659168, 0.023941604, 0.0014310129, 0.813105, 0.16126575]
['', 'frame_19', 0.00037626235, 0.025166098, 0.003252876, 0.8361095, 0.1350953]
['', 'frame_20', 0.001421177, 0.051713362, 0.02156736, 0.72886246, 0.19643566]
['', 'frame_21', 0.0017133662, 0.31597167, 0.07743658, 0.5164697, 0.08840852]
['', 'frame_22', 0.00062820397, 0.40454227, 0.15044518, 0.34785947, 0.096524805]
['', 'frame_23', 0.00083222846, 0.39282477, 0.39082283, 0.15914823, 0.05637187]
['', 'frame_24', 0.0007676377, 0.2875546, 0.4793469, 0.18273403, 0.04959685]
['', 'frame_25', 0.0005404213, 0.21532527, 0.39288023, 0.30608955, 0.08516466]
['', 'frame_26', 0.0006843791, 0.18317336, 0.351152

...

['', 'frame_16', 0.014960597, 0.00062408624, 0.00013396774, 0.98212296, 0.0021583945]
['', 'frame_17', 0.018913336, 0.00055623153, 0.000120222285, 0.9782004, 0.0022099307]
['', 'frame_18', 0.02123485, 0.00061496336, 0.000119578406, 0.97575, 0.002280574]
['', 'frame_19', 0.028189901, 0.00051925785, 0.00011735256, 0.96906817, 0.0021052468]
['', 'frame_20', 0.031510007, 0.00051284843, 0.00013906765, 0.96564084, 0.002197279]
['', 'frame_21', 0.027038783, 0.00044687593, 0.00013700163, 0.970637, 0.0017403391]
['', 'frame_22', 0.029114747, 0.00044977965, 0.00013116367, 0.96875036, 0.0015539004]
['', 'frame_23', 0.034984294, 0.00050769915, 0.00010528442, 0.9629037, 0.0014990495]
['', 'frame_24', 0.025347404, 0.00047153427, 7.64174e-05, 0.972822, 0.0012826691]
['', 'frame_25', 0.030230409, 0.00047263716, 8.606541e-05, 0.9677952, 0.0014155775]
['', 'frame_26', 0.03825633, 0.0005689049, 0.000106616746, 0.95928955, 0.001778597]
['', 'frame_27', 0.0378551, 0.0006009823, 0.00011712536, 0.95956826, 0

['', 'frame_38', 0.008875319, 1.144317e-06, 1.8064522e-07, 0.9911233, 1.1843736e-09]
['', 'frame_39', 0.38716716, 6.664152e-06, 2.6468794e-07, 0.6128258, 5.2784383e-08]
['', 'frame_40', 0.22868428, 6.9714706e-06, 1.3955305e-07, 0.7713084, 7.959019e-08]
['', 'frame_41', 0.05146624, 2.073491e-06, 5.553036e-08, 0.9485316, 2.1380215e-08]
['', 'frame_42', 0.01861007, 1.6068269e-06, 3.8047936e-08, 0.9813882, 1.8732628e-08]
['', 'frame_43', 0.01132637, 1.5722143e-06, 1.4616836e-08, 0.98867214, 2.0355687e-08]
['', 'frame_44', 0.012542808, 2.1134117e-06, 9.915976e-09, 0.987455, 2.6043473e-08]
['', 'frame_45', 0.016262876, 3.6982337e-06, 8.398314e-09, 0.98373336, 3.2157388e-08]
['', 'frame_46', 0.020352403, 4.337841e-06, 1.31631905e-08, 0.9796433, 4.7128232e-08]
['', 'frame_47', 0.054699384, 5.0261624e-06, 3.6032985e-09, 0.9452955, 1.9974149e-08]
['', 'frame_48', 0.013444839, 2.5063741e-06, 6.241697e-09, 0.98655266, 2.2742254e-08]
['', 'frame_49', 0.0030241306, 1.5477116e-07, 5.832658e-09, 0.996

['', 'frame_44', 0.0013098653, 0.00043477106, 7.395066e-06, 0.0054358905, 0.9928121]
['', 'frame_45', 0.0011127099, 0.00037839636, 3.4419452e-06, 0.010949424, 0.98755604]
['', 'frame_46', 0.0018971998, 0.0006857812, 3.7769182e-06, 0.008001069, 0.9894121]
['', 'frame_47', 0.00061960716, 0.00029861004, 2.9685812e-06, 0.0031322655, 0.9959466]
['', 'frame_48', 0.0023874608, 0.0004572496, 6.1095648e-06, 0.011438705, 0.98571044]
['', 'frame_49', 0.0016551239, 0.0004363268, 6.1785704e-06, 0.013708935, 0.9841934]
['', 'frame_50', 0.0019991798, 0.00047628817, 5.973126e-06, 0.010666411, 0.9868521]
['M10.mp4', 'frame_1', 0.012214262, 2.9263289e-08, 2.0311083e-07, 0.9877856, 1.308238e-09]
['', 'frame_2', 0.12762098, 1.909041e-08, 6.0136554e-09, 0.872379, 2.858421e-10]
['', 'frame_3', 0.2059556, 2.0965816e-08, 1.39884255e-08, 0.7940444, 4.424065e-10]
['', 'frame_4', 0.15937334, 1.21849375e-08, 1.2499974e-08, 0.8406266, 2.3150064e-10]
['', 'frame_5', 0.05019396, 1.8375292e-08, 7.5961815e-08, 0.94980

['M5.mp4', 'frame_1', 0.11112517, 0.0005497339, 9.423691e-06, 0.8821124, 0.006203299]
['', 'frame_2', 0.105801396, 0.00054136384, 8.618665e-06, 0.88702005, 0.0066286097]
['', 'frame_3', 0.07794226, 0.0004266444, 5.4398297e-06, 0.9173685, 0.0042572785]
['', 'frame_4', 0.071551554, 0.0003144061, 4.3164655e-06, 0.9248307, 0.0032991266]
['', 'frame_5', 0.071782805, 0.00031467155, 4.3063446e-06, 0.92459387, 0.0033043534]
['', 'frame_6', 0.07785729, 0.0002579828, 4.062767e-06, 0.91835237, 0.0035282269]
['', 'frame_7', 0.075905405, 0.00025841678, 3.9366932e-06, 0.92050016, 0.0033321744]
['', 'frame_8', 0.053661328, 0.00024061603, 3.178407e-06, 0.94347817, 0.0026167217]
['', 'frame_9', 0.057217807, 0.00029920196, 3.1684344e-06, 0.939754, 0.0027256836]
['', 'frame_10', 0.0611844, 0.000351452, 3.6878105e-06, 0.93523437, 0.0032261582]
['', 'frame_11', 0.06016445, 0.0003481304, 3.6928755e-06, 0.93627065, 0.0032130405]
['', 'frame_12', 0.079644285, 0.00038481894, 5.1339657e-06, 0.91543144, 0.004534

['', 'frame_14', 0.8007078, 0.047303885, 0.064982235, 0.05333494, 0.03367107]
['', 'frame_15', 0.823767, 0.042826984, 0.058938537, 0.037598904, 0.03686858]
['', 'frame_16', 0.8285511, 0.04212368, 0.04776224, 0.04384285, 0.037720162]
['', 'frame_17', 0.68706846, 0.07342624, 0.1238021, 0.08345988, 0.032243244]
['', 'frame_18', 0.6743131, 0.072130844, 0.15303074, 0.062228847, 0.038296387]
['', 'frame_19', 0.68609715, 0.06628121, 0.14358962, 0.07470462, 0.029327365]
['', 'frame_20', 0.6805546, 0.057243086, 0.15115611, 0.0728732, 0.038172975]
['', 'frame_21', 0.7242351, 0.05358077, 0.12214621, 0.06671706, 0.033320896]
['', 'frame_22', 0.7721257, 0.042393148, 0.079078734, 0.07407431, 0.032328073]
['', 'frame_23', 0.76557374, 0.04889135, 0.06791585, 0.08114876, 0.0364704]
['', 'frame_24', 0.6598707, 0.042675078, 0.13698235, 0.10660303, 0.053868875]
['', 'frame_25', 0.68255186, 0.03540221, 0.15610938, 0.080731735, 0.045204774]
['', 'frame_26', 0.68057704, 0.030176794, 0.13874306, 0.097437344, 

In [127]:
all_frame_data.head(10)

,Video,Frame No.,TALKING,HEADDOWN,YAWNING,MOBILE,NORMAL
0,T20.mp4,frame_1,0.005358,0.000824,0.00017,0.385011,0.608637
1,,frame_2,0.016445,0.001216,0.000098,0.429828,0.552413
2,,frame_3,0.028825,0.000388,0.00003,0.64578,0.324977
3,,frame_4,0.06419,0.000795,0.000025,0.203053,0.731936
4,,frame_5,0.170747,0.001998,0.000141,0.326727,0.500387
5,,frame_6,0.027918,0.001894,0.000121,0.266687,0.70338
6,,frame_7,0.014655,0.000694,0.000167,0.288295,0.696188
7,,frame_8,0.037263,0.000925,0.000175,0.261924,0.699713
8,,frame_9,0.020872,0.000736,0.000052,0.032118,0.946221
9,,frame_10,0.056808,0.001745,0.000115,0.027186,0.914146
